In [3]:
import pandas as pd
import tiktoken
import os
import json
from sklearn.model_selection import train_test_split
import fasttext
from openai import OpenAI

In [ ]:
# Load data
DATA_PATH = './data/'
df = pd.read_csv(os.path.join(DATA_PATH,'translation_data_p4.csv'), encoding='utf-8-sig')

# Set up device
api_key = "YOUR-API-KEY"
client = OpenAI(api_key = api_key)

In [23]:
df = df.dropna().copy()

In [24]:
# Add instructions column
df['instructions'] = "כתוב את הטקסט הבא בסגנון דוד בן-גוריון:\n\n"
df['input'] = df['rewritten']
df['output'] = df['paragraph']
df['input'] = df['input'].apply(lambda x: x.replace('\n\n', '\n'))
df['output'] = df['output'].apply(lambda x: x.replace('\n\n', '\n'))

print(f"Examples from file: {len(df)}")

Examples from file: 329


In [25]:
# Initialize tokenizer
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# Tokenize each document (input-output pair)
def tokenize_document(row):
    # Combine instruction + input + output into a single document
    combined_text = f"Instruction: {row['instructions']}\nInput: {row['input']}\nOutput: {row['output']}"
    tokens = encoding.encode(combined_text)
    return len(tokens), combined_text

# Apply tokenization and check token counts
df['token_count'], df['combined_text'] = zip(*df.apply(tokenize_document, axis=1))

# Check documents that exceed token limit
max_tokens = 65536
exceeds_limit = df[df['token_count'] > max_tokens]

print(f"Documents exceeding {max_tokens} tokens:\n{exceeds_limit}")


Documents exceeding 65536 tokens:
Empty DataFrame
Columns: [book_id, paragraph_num, paragraph, rewritten, instructions, input, output, token_count, combined_text]
Index: []


In [28]:
# Train/Test Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
len(train_df), len(test_df)

(263, 66)

In [29]:
def write_to_jsonl(df, filename):    
# Writing the file with UTF-8 encoding
    with open(filename, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            # Assume 'input' is the everyday version, 'output' is the original speech
            system_message = {"role": "system", "content": "אתה עוזר שכותב טקסטים בסגנון דוד בן-גוריון. תקבל טקסט יומיומי ותשכתב אותו לשפה פורמלית בסגנון בן-גוריוני."}
            
            # User and assistant messages
            user_message = {"role": "user", "content": row['input']}
            assistant_message = {"role": "assistant", "content": row['output']}

            conversation = {
            "messages": [system_message, user_message, assistant_message]
        }
            
            f.write(json.dumps(conversation, ensure_ascii=False) + "\n")
write_to_jsonl(train_df, os.path.join(DATA_PATH, 'train.jsonl'))
write_to_jsonl(test_df, os.path.join(DATA_PATH, 'test.jsonl'))

In [30]:
# Load the dataset
with open(os.path.join(DATA_PATH, 'train.jsonl'), 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 263
First example:
{'role': 'system', 'content': 'אתה עוזר שכותב טקסטים בסגנון דוד בן-גוריון. תקבל טקסט יומיומי ותשכתב אותו לשפה פורמלית בסגנון בן-גוריוני.'}
{'role': 'user', 'content': 'כל התכונות והיכולות שנדרשו בכל מקום ובכל זמן – כמו יכולת פעולה, חריצות ונאמנות – יידרשו גם לעובדי המדינה, אבל זה לא יהיה מספיק.'}
{'role': 'assistant', 'content': 'כל התכונות והסגולות שנדרשו בכל מקום ובכל זמן – כושר-פעולה, חריצות, נאמנות, – יידרשו גם לפקידות המדינה, אך אלה בלבד לא יספיקו.'}


In [31]:
from collections import defaultdict

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [32]:
import numpy as np

encoding = tiktoken.get_encoding("o200k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [33]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 65536 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 70, 1575
mean / median: 422.70722433460077, 344.0
p5 / p95: 142.8, 830.6000000000001

#### Distribution of num_assistant_tokens_per_example:
min / max: 2, 935
mean / median: 193.13307984790873, 144.0
p5 / p95: 44.0, 425.8

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


In [37]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 65536

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~111172 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~333516 tokens


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

client.files.create(
  file=open(os.path.join(DATA_PATH, 'train.jsonl'), "rb"),
  purpose="fine-tune"
)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

client.files.create(
  file=open(os.path.join(DATA_PATH, 'test.jsonl'), "rb"),
  purpose="fine-tune"
)

In [ ]:
client.fine_tuning.jobs.create(
    training_file="YOUR-FILE-NAME-HERE",
    model="gpt-4o-mini-2024-07-18",
    validation_file="YOUR-FILE-NAME-HERE",
    hyperparameters={
        "n_epochs": 3
    },
    suffix="chatDBG_v3",
)

FineTuningJob(id='ftjob-weS8qEJF8vHouHOuP8VUMZVM', created_at=1746976272, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yE4KWcntLYuSEOoxAg1h0jKj', result_files=[], seed=1832142398, status='validating_files', trained_tokens=None, training_file='file-Nye6vnnGnhMKGoKoXw2QCh', validation_file='file-NXmrfJEtzHWK7sMd3o4kHa', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=3)), type='supervised'), user_provided_suffix='chatDBG_v3', metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None)